# pyWavelet Snow Radar Processor

In [ ]:
#pyWavelet modules
from pyWavelet import snowradar, picklayers

#Python stdlib
import os
import datetime
import time

#Community packages
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
density_snow = 0.300 #Density of snow on sea ice; See Alexandrov et al., 2010, The Cryosphere
perm_snow = (1 + 0.51 * density_snow) ** 3  #Density to permitivity
n_snow = np.sqrt(perm_snow) #Permitivity to refractive index
c = 299792458 #Vacuum speed of light

In [ ]:
#radar_file = "./data/sr/Data_20160419_04_010.mat"
radar_file = "./data/eureka/sr/Data_20160419_04_025.mat"

radar_dat = snowradar.OIB(radar_file, l_case='full')

In [ ]:
radar_dat.extent

Convert the radar's GPS time to UTC and account for leap seconds

In [ ]:
example_date = datetime.datetime.utcfromtimestamp(radar_dat.time_utc[0])
print(example_date)

### Signals stuff

In [ ]:
oversample_num = 1000
num_nyquist_ts = 100
nyquist_sf = 2*radar_dat.bandwidth
fs = nyquist_sf *oversample_num 
time_step = 1/fs 
max_time  = num_nyquist_ts*oversample_num*time_step
time_vect = np.arange(-max_time,max_time,time_step)

In [ ]:
radar_dat.calcpulsewidth()
print(radar_dat.epw)
print(radar_dat.n2n)

### This applies the picklayer function to every trace
TODO: Multi thread and bake into snowradar class

In [ ]:
l_as, l_si = np.apply_along_axis(
    picklayers.picklayers, 
    0, 
    radar_dat.data_radar,
    radar_dat.n2n,
    radar_dat.dfr,
    n_snow
)

In [ ]:
snow_depth = (l_si - l_as) * radar_dat.dfr / n_snow
print("Mean snow depth (m):", np.mean(snow_depth))
plt.plot(snow_depth);

# So niceeeeeeeeeeeeee

In [ ]:
fig, ax = plt.subplots(figsize=(25,5))
ax.imshow(10 * np.log10(radar_dat.data_radar), interpolation='nearest', cmap="gist_gray")
ax.plot(l_si, color='r')
ax.plot(l_as, color='b')
ax.set_ylim([900, 600])
ax.set_aspect('auto')
fig.tight_layout();